# Who wrote this : a framework for French novelist identification

In [1]:
!pip install unidecode
!pip install transformers
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 245kB 2.7MB/s 
     |████████████████████████████████| 542kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 12.9MB/s 
     |████████████████████████████████| 1.0MB 18.2MB/s 
     |████████████████████████████████| 870kB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e3fbec19081189c2f33c9d3b29b9cca56138eee06efd0da1ccdd5886bfa201f5
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 88.6MB 5.0MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338488 sha256=cf44a8aff8a2bdbb875bcbf24a74442c5354a49b8b47dc3bb373886859bbf59b
  Stored in directory: /tmp/pip-ephem-wheel-cache-zauex4ds/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation success

In [0]:
import os
import re
import random
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count

import unidecode
import urllib.request
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, confusion_matrix
import spacy
import fr_core_news_md
import gensim
from gensim.models import Doc2Vec, FastText
from gensim.models.doc2vec import TaggedDocument
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from transformers import AutoModel, AutoTokenizer

In [0]:
# Number of available cores for parallel computing
N_CORES = cpu_count()

## Data loading

In [4]:
# Download data from the GitHub repository
!wget https://raw.githubusercontent.com/meteve/NLP_project/master/data/corpus_train.csv
!wget https://raw.githubusercontent.com/meteve/NLP_project/master/data/corpus_test.csv

--2020-03-27 20:34:09--  https://raw.githubusercontent.com/meteve/NLP_project/master/data/corpus_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18165744 (17M) [text/plain]
Saving to: ‘corpus_train.csv’

corpus_train.csv    100%[===================>]  17.32M   107MB/s    in 0.2s    

2020-03-27 20:34:11 (107 MB/s) - ‘corpus_train.csv’ saved [18165744/18165744]

--2020-03-27 20:34:14--  https://raw.githubusercontent.com/meteve/NLP_project/master/data/corpus_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6894810 (6.6M) [te

In [0]:
# Import train data
train_df = pd.read_csv('corpus_train.csv', sep='|')
train_df = train_df.sample(frac=1).reset_index(drop=True) # Shuffle
X_train = train_df['paragraph'].values
y_labels_train = train_df['author'].values

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_labels_train)
N_CLASSES = len(np.unique(y_train))

In [0]:
# Import test data and build validation dataset
test_df = pd.read_csv('corpus_test.csv', sep='|')
test_df = test_df.sample(frac=1).reset_index(drop=True) # Shuffle
X_val, X_test, y_val, y_test = train_test_split(test_df['paragraph'].values,
                                                test_df['author'].values,
                                                test_size=0.5, random_state=42)
y_val = le.transform(y_val)
y_test = le.transform(y_test)

In [8]:
train_df.head(5)

,paragraph,author
0,"--Je vais retirer l'échelle, dit-il, pour qu'e...",Stendhal
1,Tout le petit bois conspire pour l'empêcher de...,Daudet
2,Frédéric passa la sienne trois jours après. Av...,Flaubert
3,"-- Oui, reprit Porthos en se ravisant; mais un...",Dumas
4,«Renvoyer le comte... me trouver seule avec lu...,Stendhal


# Training

### Baseline : TF-IDF

In [0]:
URL = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-fr/master/stopwords-fr.txt'
response = urllib.request.urlopen(URL)
stopwords = response.read().decode('utf-8').splitlines()
stopwords = [unidecode.unidecode(x) for x in stopwords]
stopwords.append('quelqu') # Make stopwords consistent with scikit tokenizer

In [0]:
# ML pipeline : TF-IDF + SVM classifier

tfidf_vecto = TfidfVectorizer(stop_words=stopwords)
clf = LogisticRegression(max_iter=10000)

tfidf_pipeline = Pipeline([
                           ('tf-idf', tfidf_vecto),
                           ('clf', clf)
])

In [0]:
# Keep sklearn preprocessing pipeline for later
preprocessor = tfidf_vecto.build_analyzer()

In [0]:
# Preprocessing + training
tfidf_pipeline = tfidf_pipeline.fit(X_train, y_train)

In [13]:
# Compute predictions and validation score
y_val_pred_tfidf = tfidf_pipeline.predict(X_val)
tfidf_val_score = f1_score(y_val, y_val_pred_tfidf, average='micro')
print('F1 score on validation set with TF-IDF :', 
      tfidf_val_score.round(2))

F1 score on validation set with TF-IDF : 0.48


In [15]:
# Compute predictions and score on train set
y_train_pred_tfidf = tfidf_pipeline.predict(X_train)
tfidf_train_score = f1_score(y_train, y_train_pred_tfidf, average='micro')
print('F1 score on train set with TF-IDF :', 
      tfidf_train_score.round(2))

F1 score on train set with TF-IDF : 0.92


The train score indicates that the baseline overfits the training data. It may be due to the fact that there are different books in the train and test sets. Some locations and characters which are present in the books of the training set may have a strong influence on the training of the model. We first compute tf-idf weights for the well classified paragraphs of the train set, and look at the most important words for each author in terms of tf-idf weights.  

In [0]:
def top_tfidf_feats(row, features, top_n):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [0]:
def get_tfidf_weights_by_author(author, NER=False, top_n=20):
    '''Return words with high tfidf weights, for well-classified paragraphs of an author with or without NER'''
    y_true = y_train
    
    if NER == True:
        y_pred = y_train_pred_tfidf_ner
        pipe = tfidf_pipeline_ner
        X = X_train_ner
    else:
        y_pred = y_train_pred_tfidf
        pipe = tfidf_pipeline
        X = X_train
        
    
    if type(author) == str:
        author_index = list(le.classes_).index(author)
    elif type(author) == int:
        author_index = author
   
    # keep well classified paragraphs of the auhtor
    well_classif_indexes = ((y_true == author_index) & (y_pred == author_index))
    X_train_well_classif = X[well_classif_indexes]
    
    # get tf-idf vectors
    vec = tfidf_pipeline.named_steps['tf-idf']
    Xtr = vec.transform(X_train_well_classif)
    features = vec.get_feature_names()
    
    # get mean tf-idf scores
    D = Xtr.toarray()
    tfidf_means = np.mean(D, axis=0)
    
    return top_tfidf_feats(tfidf_means, features, top_n)

In [0]:
def get_tfidf_weights_by_author(author, y_true, y_pred, X, top_n=20):
    '''Return words with high tfidf weights, for well-classified paragraphs of an author with or without NER'''        
    
    if type(author) == str:
        author_index = list(le.classes_).index(author)
    elif type(author) == int:
        author_index = author
   
    # keep well classified paragraphs of the auhtor
    well_classif_indexes = ((y_true == author_index) & (y_pred == author_index))
    X_train_well_classif = X[well_classif_indexes]
    
    # get tf-idf vectors
    vec = tfidf_pipeline.named_steps['tf-idf']
    Xtr = vec.transform(X_train_well_classif)
    features = vec.get_feature_names()
    
    # get mean tf-idf scores
    D = Xtr.toarray()
    tfidf_means = np.mean(D, axis=0)
    
    return top_tfidf_feats(tfidf_means, features, top_n)

In [31]:
get_tfidf_weights_by_author(author='Maupassant', y_true=y_train, y_pred=y_train_pred_tfidf, X=X_train)

,feature,tfidf
0,jeanne,0.033423
1,duroy,0.024628
2,walter,0.014932
3,était,0.013707
4,forestier,0.012990
5,julien,0.012916
6,mme,0.011623
7,femme,0.011069
8,ça,0.010379
9,georges,0.009579


A relatively high number of proper nouns have a high tf-idf weight in the previous model. That's why we implement a Named Entity Recognition (NER) procedure and evaluate whether it reduces the overfitting. 

### NER

In [0]:
nlp = fr_core_news_md.load()

In [0]:
def remove_named_entities(paragraph): 
  """remove the named entities from a paragraph"""
  doc = nlp(paragraph)
  names = []
  for ent in doc.ents:
      # list of all the named entities recognized in the paragraph
      names.append(ent)
  # keep unique elements and convert to string    
  names = set([str(x) for x in names])
    
  # remove named entities from the paragraph
  paragraph_no_names = paragraph
  for name in names:
      paragraph_no_names = paragraph_no_names.replace(name, '')
        
  return paragraph_no_names

In [25]:
X_train_ner = []

for i, par in enumerate(X_train):
  par_ner = remove_named_entities(par)
  X_train_ner.append(par_ner)
  if i%1000 == 0:
        print(i)

X_train_ner = np.array(X_train_ner)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000


In [26]:
X_val_ner = []

for i, par in enumerate(X_val):
  par_ner = remove_named_entities(par)
  X_val_ner.append(par_ner)
  if i%1000 == 0:
        print(i)

X_val_ner = np.array(X_val_ner)

0
1000
2000
3000
4000
5000
6000
7000
8000


In [0]:
_ = tfidf_pipeline.fit(X_train_ner, y_train)

In [28]:
# Compute predictions and score on train set AFTER NER procedure
y_train_pred_tfidf_ner = tfidf_pipeline.predict(X_train_ner)
tfidf_train_score_ner = f1_score(y_train, y_train_pred_tfidf_ner, average='micro')
print('F1 score on train set with TF-IDF after NER:', 
      tfidf_train_score_ner.round(2))

F1 score on train set with TF-IDF after NER: 0.82


The train score is much lower after removing the proper nouns in the paragraphs. The overfitting was partially due to the presence of such proper nouns. 

In [29]:
# Compute predictions and validation score AFTER NER procedure
y_val_pred_tfidf_ner = tfidf_pipeline.predict(X_val_ner)
tfidf_val_score_ner = f1_score(y_val, y_val_pred_tfidf_ner, average='micro')
print('F1 score on validation set with TF-IDF after NER:', 
      tfidf_val_score_ner.round(2))

F1 score on validation set with TF-IDF after NER: 0.42


The validation score is lower when we remove the proper nouns of the paragraphs. This suggests that some names may appear in different books of the same author, and can be helpful for the predictions. We explore the words that have the highest tf-idf weights after NER procedure, even if we choose to keep the proper nouns in the paragraphs in the following models. 

In [32]:
get_tfidf_weights_by_author(author='Maupassant', y_true=y_train, y_pred=y_train_pred_tfidf_ner, X=X_train_ner)

,feature,tfidf
0,était,0.012964
1,femme,0.011081
2,point,0.010577
3,là,0.008892
4,bras,0.008730
5,mère,0.008695
6,baron,0.008583
7,où,0.008556
8,semblait,0.008394
9,petite,0.008137


This analysis enables us to check whether the proper nouns were removed in the NER procedure. 

### FastText (averaging of pre-trained word vectors)

In [34]:
# Download and extract FastText French word vectors
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz
!gunzip cc.fr.300.bin.gz

--2020-03-27 21:05:59--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4b8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4496886212 (4.2G) [application/octet-stream]
Saving to: ‘cc.fr.300.bin.gz’

cc.fr.300.bin.gz    100%[===================>]   4.19G  11.4MB/s    in 6m 26s  

2020-03-27 21:12:27 (11.1 MB/s) - ‘cc.fr.300.bin.gz’ saved [4496886212/4496886212]



In [0]:
# Import Fasttext French word vectors
fasttext = FastText.load_fasttext_format('cc.fr.300.bin')

In [0]:
def text_to_wv_fasttext(text):
    """Compute average of FastText's word vectors for a given text."""
    if text:
        tokens = preprocessor(text)
        wv_mat = np.zeros((len(tokens), fasttext.vector_size))
        for i, tok in enumerate(tokens):
            try:
                wv_mat[i] = fasttext.wv[tok]
            except KeyError:
                pass
        text_vec = wv_mat.mean(axis=0)
    else:
        text_vec = np.zeros(fasttext.vector_size)
    return text_vec

In [0]:
def preprocess_corpus_fasttext(corpus):
    """Parallelize preprocessing and document vectors computation."""
    with Pool(N_CORES) as p:
        corpus_prepro = p.map(text_to_wv_fasttext, list(corpus))
    return np.array(corpus_prepro)

In [0]:
class TextToWV(BaseEstimator, TransformerMixin):
    """Enable to use preprocessing function in a sklearn pipeline."""
    def __init__(self, preprocessor):
        self.preprocessor = preprocessor

    def fit(self, X, y=None):
        return(self)

    def transform(self, X):
        return self.preprocessor(X)

In [0]:
# Prediction pipeline
clf = LogisticRegression(max_iter=10000)
fasttext_pipeline = Pipeline([
                              ('fasttext_average', TextToWV(preprocess_corpus_fasttext)),
                              ('clf', clf)
])

In [0]:
# Preprocessing + training
fasttext_pipeline = fasttext_pipeline.fit(X_train, y_train)

In [42]:
# Compute predictions and test score
y_val_pred_fasttext = fasttext_pipeline.predict(X_val)
val_score_fasttext = f1_score(y_val, y_val_pred_fasttext, average='micro')
print('F1 score on validation set :',
      val_score_fasttext.round(2))

F1 score on validation set : 0.33


### Doc2Vec

In [0]:
def build_d2v_corpus(corpus, tokens_only=False):
    """Tokenize and build corpus as expected by Gensim Doc2Vec class."""
    corpus_tokenized = []
    for i, text in enumerate(corpus):
        tokens = preprocessor(text)
        if tokens_only:
            corpus_tokenized.append(tokens)
        else:
            corpus_tokenized.append(TaggedDocument(tokens, [i]))
    return corpus_tokenized

In [0]:
# Format train and validation corpus as required by Doc2Vec
corpus_train_d2v = build_d2v_corpus(X_train, tokens_only=False)
corpus_val_d2v = build_d2v_corpus(X_val, tokens_only=True)

In [0]:
# Train Doc2Vec model
model = Doc2Vec(vector_size=50, min_count=2, epochs=10, workers=N_CORES)
model.build_vocab(corpus_train_d2v)
model.train(corpus_train_d2v, total_examples=model.corpus_count, 
            epochs=model.epochs)

In [0]:
# Compute document vectors on train and validation sets
X_train_d2v = np.array([model.infer_vector(doc.words) for doc in corpus_train_d2v])
X_val_d2v = np.array([model.infer_vector(doc) for doc in corpus_val_d2v])

In [47]:
# Compute predictions and validation score
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train_d2v, y_train)
y_val_pred_d2v = clf.predict(X_val_d2v)
val_score_d2v = f1_score(y_val, y_val_pred_d2v, average='micro')
print('F1 score on validation set :', 
      val_score_d2v.round(2))

F1 score on validation set : 0.37


### CamemBERT

In [0]:
class CorpusToTorchDataset(Dataset):
    """Convert corpus to tensors of token indices in CamemBERT vocabulary."""
    def __init__(self, corpus, labels, model_name, maxlen=100):
        self.corpus = corpus
        self.labels = labels
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.maxlen = maxlen

    def __len__(self):
        return len(self.corpus)

    def __getitem__(self, index):

        # Select instance
        sentence = self.corpus[index]
        label = self.labels[index]

        # Preprocess data as required by BERT models
        tokens = self.tokenizer.tokenize(sentence)
        bos_token = self.tokenizer.cls_token
        eos_token = self.tokenizer.sep_token
        pad_token = self.tokenizer.pad_token
        # Insert CLS and SEP tokens at beginning and end of sentence
        tokens = [bos_token] + tokens + [eos_token]
        if len(tokens) < self.maxlen:
            # If sentence is shorter than maxlen, pad sentence using special 
            # padding token
            tokens = tokens + [pad_token for _ in range(self.maxlen - len(tokens))]
        else:
            # Cut the sentence if it is longer than maxlen
            tokens = tokens[:self.maxlen-1] + [eos_token]

        # Convert tokens to tensor of indices in CamemBERT vocabulary
        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        tokens_ids_tensor = torch.tensor(tokens_ids)
        # Get attention mask to distinguish padding tokens from actual tokens
        pad_token_id = self.tokenizer.convert_tokens_to_ids(pad_token)
        attn_mask = (tokens_ids_tensor != pad_token_id).long()

        return tokens_ids_tensor, attn_mask, label

In [56]:
# Create instances of training and validation dataloaders

BERT_MODEL_NAME = 'camembert-base'
MAXLEN = 100
BATCH_SIZE = 12

train_set = CorpusToTorchDataset(X_train, y_train, model_name=BERT_MODEL_NAME, maxlen=100)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, num_workers=N_CORES)

val_set = CorpusToTorchDataset(X_val, y_val, model_name=BERT_MODEL_NAME, maxlen=100)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, num_workers=N_CORES)

In [0]:
class CamemBERTClassifier(nn.Module):
    """Perform fine-tuning and classification using CamemBERT."""
    def __init__(self, pretrained_model_name=BERT_MODEL_NAME):
        super(CamemBERTClassifier, self).__init__()
        # Load CamemBERT
        self.encoder = AutoModel.from_pretrained(pretrained_model_name)
        # Add an extra dense layer to perform classification
        self.cls_layer = nn.Linear(self.encoder.pooler.dense.out_features, N_CLASSES)

    def forward(self, seq, attn_masks):
        # Feed input to BERT model to obtain contextualized representations
        cont_reps, _ = self.encoder(seq, attention_mask = attn_masks)
        # Get representation of [CLS] head
        cls_rep = cont_reps[:, 0]
        # Feed document representation to the classifying layer
        logits = self.cls_layer(cls_rep)

        return logits

In [58]:
# Instantiate CamemBERT classifier model
camembert_clf = CamemBERTClassifier()

In [0]:
# Define loss and optimizer
criterion = CrossEntropyLoss()
opti = Adam(camembert_clf.parameters(), lr = 3e-5)

In [0]:
def train(model, criterion, opti, train_loader, val_loader, max_eps=3, 
          gpu=True, print_every=100, validate_every=1):
    """Train a transformer model and compute loss on validation data."""
    if gpu:
        model = model.to("cuda")
    # Unfreeze weights to allow fine tuning
    model.train() 

    train_loss_total = 0
    n_batch_train = 0
    for ep in range(max_eps):
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            # Clear gradients (avoid accumulation)
            opti.zero_grad()  
            # Transfer tensors to GPU
            if gpu:
                seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()
            # Compute logits
            logits = model(seq, attn_masks)
            # Compute batch loss
            loss = criterion(logits, labels)
            # Backpropagation
            loss.backward()
            # Optimization step
            opti.step()
            # Accumulate train loss over batches
            train_loss_total += loss.item()
            n_batch_train += 1

            # Compute average loss over the last `print_every` training batches
            if print_every is not None and (it + 1) % print_every == 0:
                print(f'Epoch {ep+1}, batch {it+1}. Average loss over last {print_every} training batches : {train_loss_total/n_batch_train}')
                # Reinitialize accumulators
                train_loss_total = 0
                n_batch_train = 0

        if validate_every is not None and ep % validate_every == 0:
            # Evaluation on the validation set

            predictions_val = []
            true_labels_val = []
            for it, (seq, attn_masks, labels) in enumerate(val_loader):
                if gpu:
                    seq, attn_masks = seq.cuda(), attn_masks.cuda()
                # Compute logits without constructing the computing graph
                # (only needed for backprop)
                with torch.no_grad():
                    logits_val = model(seq, attn_masks)
                preds_batch = torch.argmax(logits_val, 1).cpu().numpy()
                predictions_val.extend(preds_batch)
                true_labels_val.extend(labels.numpy())

            val_f1 = f1_score(true_labels_val, predictions_val, average='micro')
            print('------------------------------------------------------------')
            print("Epoch {} complete. F1 score on validation data : {}".format(ep+1, val_f1))
            print('------------------------------------------------------------')

In [61]:
# Enforce deterministic behavior to ensure reproducibility of the results
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

train(camembert_clf, criterion, opti, train_loader, val_loader,
      max_eps=1, gpu=True, print_every=500, validate_every=1)

# Grid search (only 1 epoch each time, systematic overtraining above that) :
# lr = 5e-5 => val_f1 = 0.49
# lr = 3e-5 => val_f1 = 0.52

Epoch 1, batch 500. Average loss over last 500 training batches : 1.5918752009868622
Epoch 1, batch 1000. Average loss over last 500 training batches : 0.9780670761466026
Epoch 1, batch 1500. Average loss over last 500 training batches : 0.7167568392157555
Epoch 1, batch 2000. Average loss over last 500 training batches : 0.623441261216998
Epoch 1, batch 2500. Average loss over last 500 training batches : 0.5532458843886853
Epoch 1, batch 3000. Average loss over last 500 training batches : 0.4762355045974255
Epoch 1, batch 3500. Average loss over last 500 training batches : 0.4380950675457716
------------------------------------------------------------
Epoch 1 complete. F1 score on validation data : 0.4647969960103262
------------------------------------------------------------


# Final evaluation

### Quantitative evaluation: F1 score

In [0]:
# TFIDF

y_test_pred_tfidf = tfidf_pipeline.predict(X_test)
test_score_tfidf = f1_score(y_test, y_test_pred_tfidf, average='micro')

In [0]:
# FastText

y_test_pred_fasttext = fasttext_pipeline.predict(X_test)
test_score_fasttext = f1_score(y_test, y_test_pred_fasttext, average='micro')

In [0]:
# doc2vec

corpus_test_d2v = build_d2v_corpus(X_test, tokens_only=True)
X_test_d2v = np.array([model.infer_vector(doc) for doc in corpus_test_d2v])
y_test_pred_d2v = clf.predict(X_test_d2v)
test_score_d2v = f1_score(y_test, y_test_pred_d2v, average='micro')

In [0]:
# CamemBERT

# Create instance of test dataloader
test_set = CorpusToTorchDataset(X_test, y_test, model_name=BERT_MODEL_NAME, maxlen=100)
test_loader = DataLoader(test_set, batch_size = 12, num_workers = N_CORES)

def eval(model, test_loader, gpu=True):
    """Compute predictions of a trained transforme model on test data."""
    # Put model in evaluation mode
    model.eval()

    predictions = []
    for it, (seq, attn_masks, labels) in enumerate(test_loader):
        if gpu:
            seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()
        # Compute logits without constructing the computing graph
        # (only needed for backprop)
        with torch.no_grad():
            logits = model(seq, attn_masks)
        # Compute predictions from logits and store them as arrays
        preds_batch = torch.argmax(logits, 1).cpu().numpy()
        predictions.extend(preds_batch)

    return predictions

y_test_pred_camembert = eval(camembert_clf, test_loader)
test_score_camembert = f1_score(y_test, y_test_pred_camembert, average='micro')

In [66]:
# Compare performances
models = ['TF-IDF', 'FastText', 'Doc2Vec', 'CamemBERT']
scores = [test_score_tfidf, test_score_fasttext, test_score_d2v, test_score_camembert]
df_scores = pd.DataFrame(zip(models, scores), columns=['Model', 'F1 score'])
df_scores.sort_values('F1 score', ascending=False)

,Model,F1 score
3,CamemBERT,0.485862
0,TF-IDF,0.441159
2,Doc2Vec,0.374399
1,FastText,0.340960


We find that CamemBERT has the highest F1 score. In order to make a qualitative evaluation of this model, we build the confusion matrix based on the predictions of CamemBERT on the test set.

### Qualitative evaluation: confusion matrix

In [0]:
def get_conf_matrix_df(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred)
    conf_matrix = pd.DataFrame(conf_matrix, columns=list(le.classes_), index=list(le.classes_))
    
    row_percentages = []
    for i in range(0,10):
        perc = round(conf_matrix.iloc[i,i]/sum(conf_matrix.iloc[0:10,i]), 3)
        row_percentages.append(perc)
    
    row_percentages = pd.DataFrame([row_percentages], columns=list(le.classes_))
    
    col_percentages = []
    for i in range(0,10):
        perc = round(conf_matrix.iloc[i,i]/sum(conf_matrix.iloc[i,0:10]), 3)
        col_percentages.append(perc)
    col_percentages.append(None)
    
    conf_matrix = conf_matrix.append(row_percentages)
    conf_matrix = conf_matrix.rename(index={0: 'PRECISION'})
    conf_matrix['RECALL'] = col_percentages

    return(conf_matrix)

In [0]:
conf_matrix_BERT = get_conf_matrix_df(y_test, y_test_pred_camembert)

In [69]:
conf_matrix_BERT

,Balzac,Daudet,Dumas,Flaubert,Hugo,Maupassant,Stendhal,Verne,Vigny,Zola,RECALL
Balzac,202.000,33.000,68.000,36.000,24.000,13.000,392.00,258.000,34.000,10.000,0.189
Daudet,0.000,63.000,27.000,55.000,19.000,31.000,4.00,149.000,5.000,1.000,0.178
Dumas,18.000,14.000,2008.000,49.000,61.000,32.000,50.00,19.000,53.000,0.000,0.872
Flaubert,3.000,8.000,14.000,506.000,23.000,46.000,13.00,5.000,6.000,0.000,0.811
Hugo,4.000,11.000,39.000,72.000,223.000,8.000,1050.00,47.000,97.000,0.000,0.144
Maupassant,9.000,51.000,34.000,26.000,35.000,96.000,13.00,27.000,6.000,3.000,0.320
Stendhal,1.000,1.000,39.000,11.000,20.000,14.000,542.00,3.000,5.000,0.000,0.852
Verne,0.000,3.000,62.000,21.000,21.000,5.000,10.00,290.000,8.000,0.000,0.690
Vigny,22.000,61.000,23.000,14.000,24.000,1.000,35.00,26.000,54.000,0.000,0.208
Zola,61.000,83.000,236.000,150.000,31.000,84.000,58.00,92.000,52.000,157.000,0.156


!! CONFUSION MATRIX A INTERPRETER !!